In [1]:
import tensorflow as tf
DATA_PATH = 'D:\kaggle\visual_relationship'

In [2]:
def conv_layer(input_X, filters, kernel_size=5, pool_size=1, name="conv"):
    with tf.name_scope(name):
        channels_in = int(input_X.shape[-1])
        w = tf.Variable(tf.truncated_normal([kernel_size, channels_in, filters], stddev=0.1), name='w')
        b = tf.Variable(tf.constant(0.1, shape=[filters], name="b"))
        conv = tf.nn.conv1d(input_X, w, stride=1, padding='SAME')
        act = tf.nn.relu(conv + b)

        return tf.layers.max_pooling1d(act, pool_size=pool_size, strides=pool_size, padding="SAME")

In [6]:
def fc_layer(input_X, output_size, name="fc"):
    with tf.name_scope(name):
        input_size = int(input_X.shape[-1])
        w = tf.Variable(tf.truncated_normal([input_size, output_size], stddev=0.1), name='w')
        b = tf.Variable(tf.constant(0.1, shape=[output_size], name="b"))
        act = tf.nn.sigmoid(tf.matmul(input_X, w) + b)

        return act

In [7]:
def lstm_cell(num_cell, keep_prob):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_units=num_cell, state_is_tuple=True)
    return tf.contrib.rnn.DropoutWrapper(cell=lstm_cell, output_keep_prob=keep_prob)

In [8]:
def build_model(temp_dataset, req_config):
    with tf.device('/cpu:0'):
        keep_prob = tf.placeholder(tf.float32, [])
        batch_size = tf.placeholder(tf.int32, [])
        learning_rate = float(req_config['learning_rate']) if 'learning_rate' in req_config.keys() else 0.001

        data_layer = init_input_tensor(temp_dataset)
        hidden_layer = init_hidden_tensor(data_layer, keep_prob, batch_size)
        output_layer = init_output_tensor(data_layer, hidden_layer)

        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=data_layer['group_label'],
                                                                      logits=output_layer))
        correct_prediction = tf.equal(tf.argmax(output_layer, 1), tf.argmax(data_layer['group_label'], 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        gradients = optimizer.compute_gradients(loss=cost)
        train_op = optimizer.apply_gradients(gradients)
        l2_norm = lambda t: tf.sqrt(tf.reduce_sum(tf.pow(t, 2)))
        
        init = tf.global_variables_initializer()
        return config

In [9]:
def batch_norm(x):
    epsilon = 1e-3
    batch_mean, batch_var = tf.nn.moments(x, [0, 1, 2])
    return tf.nn.batch_normalization(x, batch_mean, batch_var, offset=None, scale=None, variance_epsilon=epsilon)

In [10]:
def init_model(sess, config, reload):
    sess.run(config['init'])
    if reload == 'True':
        saver = tf.train.Saver()
        path = DATA_PATH + 'result/model/' + config['model_name'] + '.ckpt'
        saver.restore(sess, path)
        print("=== Model restored ===")

In [1]:
def save_model(sess, config, valid_acc, num_step, db_conn):
    saver = tf.train.Saver()
    path = DATA_PATH + 'result/model/' + config['model_name'] + '.ckpt'
    saver.save(sess, path)

    num_params = count_parameters()
    sql4model = sql4Tensorflow(model_name=config['model_name'], customer_group=-1, creator=config['user'],
                               sql_conn=db_conn)
    sql4model.save2sql(model_type=config['model_type'], model_params=num_params, valid_acc=valid_acc, step=num_step)
    print("=== Model saved ===")